In [5]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

In [6]:
import pennylane as qml
from pennylane import numpy as np

In [7]:
import torch
import torch.nn as nn

## SMILES to QMSE-Compatible Molecular Data
To proceed with starting from SMILES strings and converting them into the detailed molecular representations required for Quantum Molecular Structure Encoding (QMSE), here is a step-by-step approach with Python usage leveraging RDKit:

1. Input Canonical SMILES
2. Generate 3D Conformers for accurate geometry
3. Extract bond orders and stereochemistry
4. Calculate Isomerism Parameters `epsilon_D` and `epsilon_T`
5. Construct Hybrid Coulomb-Adjacency Matrix

In [8]:
df = pd.read_csv('qm9.csv')
df.head()

,mol_id,smiles,A,B,C,mu,alpha,homo,lumo,gap,...,zpve,u0,u298,h298,g298,cv,u0_atom,u298_atom,h298_atom,g298_atom
0,gdb_1,C,157.71180,157.709970,157.706990,0.0000,13.21,-0.3877,0.1171,0.5048,...,0.044749,-40.478930,-40.476062,-40.475117,-40.498597,6.469,-395.999595,-398.643290,-401.014647,-372.471772
1,gdb_2,N,293.60975,293.541110,191.393970,1.6256,9.46,-0.2570,0.0829,0.3399,...,0.034358,-56.525887,-56.523026,-56.522082,-56.544961,6.316,-276.861363,-278.620271,-280.399259,-259.338802
2,gdb_3,O,799.58812,437.903860,282.945450,1.8511,6.31,-0.2928,0.0687,0.3615,...,0.021375,-76.404702,-76.401867,-76.400922,-76.422349,6.002,-213.087624,-213.974294,-215.159658,-201.407171
3,gdb_4,C#C,0.00000,35.610036,35.610036,0.0000,16.28,-0.2845,0.0506,0.3351,...,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,-385.501997,-387.237686,-389.016047,-365.800724
4,gdb_5,C#N,0.00000,44.593883,44.593883,2.8937,12.99,-0.3604,0.0191,0.3796,...,0.016601,-93.411888,-93.409370,-93.408425,-93.431246,6.278,-301.820534,-302.906752,-304.091489,-288.720028


In [9]:
def QMSEMatrix(smiles):
    mol = Chem.MolFromSmiles(smiles, sanitize=True)
    mol = Chem.AddHs(mol)
    Chem.AssignStereochemistry(mol, cleanIt=True, force=True)
    AllChem.EmbedMolecule(mol, randomSeed = 67)
    AllChem.MMFFOptimizeMolecule(mol)
    conf = mol.GetConformer()
    #coords = [conf.GetAtomPosition(i) for i in range(mol.GetNumAtoms())]
    bond_orders_and_stereo = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        order = bond.GetBondTypeAsDouble()
        stereo = Chem.BondStereo.STEREONONE
        epsilon_D = 1
        if bond.GetBondType() == Chem.BondType.DOUBLE and bond.GetStereo() == Chem.BondStereo.STEREOZ:
            epsilon_D = -1
        bond_orders_and_stereo.append((i, j, order, epsilon_D))
    chiral_centers = Chem.FindMolChiralCenters(mol, includeUnassigned=True)
    epsilon_T = {}
    for atom_idx, chirality in chiral_centers:
        if chirality == Chem.CHIRALITY_S:
            epsilon_T[atom_idx] = -1
        else:
            epsilon_T[atom_idx] = 1
    num_atoms = mol.GetNumAtoms()
    M = np.zeros((num_atoms, num_atoms))
    Z = [atom.GetAtomicNum() for atom in mol.GetAtoms()]

    for i in range(num_atoms):
        M[i, i] = 0.5 * epsilon_T.get(i, 1) * (Z[i] ** 3)

    for i, j, order, epsilon_D in bond_orders_and_stereo:
        M[i, j] = M[j, i] = (epsilon_D * Z[i] * Z[j]) / order
        
    return M, bond_orders_and_stereo

In [10]:
QMSEMatrix(df.iloc[16]['smiles'])

(tensor([[108. ,  36. ,  48. ,   6. ,   6. ,   0. ,   0. ],
         [ 36. , 108. ,  48. ,   0. ,   0. ,   6. ,   6. ],
         [ 48. ,  48. , 256. ,   0. ,   0. ,   0. ,   0. ],
         [  6. ,   0. ,   0. ,   0.5,   0. ,   0. ,   0. ],
         [  6. ,   0. ,   0. ,   0. ,   0.5,   0. ,   0. ],
         [  0. ,   6. ,   0. ,   0. ,   0. ,   0.5,   0. ],
         [  0. ,   6. ,   0. ,   0. ,   0. ,   0. ,   0.5]], requires_grad=True),
 [(0, 1, 1.0, 1),
  (1, 2, 1.0, 1),
  (2, 0, 1.0, 1),
  (0, 3, 1.0, 1),
  (0, 4, 1.0, 1),
  (1, 5, 1.0, 1),
  (1, 6, 1.0, 1)])

## Quantum Circuit Encoding of the QMSE + MP

In [48]:
def FixedQMSECircuit(
    matrix,
    edge_list,
    params,
    n_qubits: int,
    n_layers: int,
    dev=None
):
    if not dev: dev = qml.device('default.qubit', wires=n_qubits)

    @qml.qnode(dev, interface='torch')
    def applyCircuit():
        # Initial QMSE encoding - RY rotations per node
        for i in range(n_qubits):
            qml.RY(matrix[i, i], wires=i) 
        
        # Initial bond IsingXX rotations
        for i in range(n_qubits):
            for j in range(i+1, n_qubits):
                if matrix[i, j] != 0:
                    qml.IsingXX(matrix[i, j], wires=[i,j])
        
        # Message passing layers
        for layer in range(n_layers):
            for edge_idx, (i, j, _, _) in enumerate(edge_list):
                qml.IsingXX(params[layer, edge_idx], wires=[i, j])
            for i in range(n_qubits):
                qml.RY(params[layer, i + len(edge_list)], wires=i)
        
        # Collect per-node features (2 observables per qubit: Z, X)
        node_feats = []
        for i in range(n_qubits):
            node_feats.append(qml.expval(qml.PauliZ(i)))
            node_feats.append(qml.expval(qml.PauliX(i)))
        
        # Collect per-bond features (Z_i Z_j correlators)
        bond_feats = []
        for i, j, _, _ in edge_list:
            bond_feats.append(qml.expval(qml.PauliZ(i) @ qml.PauliZ(j)))
        return node_feats, bond_feats   # tuple of two tensors

    return applyCircuit

In [ ]:
mysteryMatrix, mysteryEL = QMSEMatrix(df.iloc[16]['smiles'])
n_qubits = len(mysteryMatrix)
n_layers = 4 # based on lit, 3-5 provides the best results
num_params = len(mysteryEL) + n_qubits
mysteryWeight = torch.randn(n_layers, num_params, requires_grad=True)
mysteryCircuit = FixedQMSECircuit(
    mysteryMatrix,
    mysteryEL,
    mysteryWeight,
    n_qubits,
    n_layers
)

In [43]:
nodeFeatures, bondFeatures = mysteryCircuit()

/Users/maxcui/Downloads/ElectroDragon/coconut/lib/python3.11/site-packages/pennylane/math/interface_utils.py:136: UserWarning: Contains tensors of types {'autograd', 'torch'}; dispatch will prioritize TensorFlow, PyTorch, and Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(


In [44]:
nodeFeatures = torch.stack(nodeFeatures).reshape(n_qubits, 2).to(torch.float32)   # [n_qubits, 2]
bondFeatures = torch.stack(bondFeatures).reshape(len(mysteryEL), 1).to(torch.float32)  # [n_bonds, 1]

In [45]:
bondFeatures

tensor([[ 0.0549],
        [-0.0833],
        [-0.4017],
        [-0.3263],
        [ 0.3957],
        [ 0.2802],
        [ 0.5132]], grad_fn=<ToCopyBackward0>)

In [46]:
nodeFeatures

tensor([[-0.4368, -0.1431],
        [-0.0837,  0.0275],
        [ 0.9042,  0.0248],
        [ 0.5308, -0.5608],
        [-0.9088, -0.2330],
        [-0.3431, -0.0837],
        [-0.3344,  0.3946]], grad_fn=<ToCopyBackward0>)

## Classical Multi-Layer Perceptron

In [39]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, feature_dim, hidden_dim=64, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.SiLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LayerNorm(hidden_dim // 2),
            nn.SiLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 1)
        )
    
    def forward(self, node_feats, bond_feats):
        """
        node_feats: tensor [n_nodes, F_node]
        bond_feats: tensor [n_bonds, F_bond]
        returns: tensor [1] predicted property
        """
        # Compute mean, max, std pooling along node/bond dimension (dim=0)
        n_mean = node_feats.mean(dim=0)
        n_max  = node_feats.max(dim=0).values
        n_std  = node_feats.std(dim=0)

        b_mean = bond_feats.mean(dim=0)
        b_max  = bond_feats.max(dim=0).values
        b_std  = bond_feats.std(dim=0)

        # Concatenate all pooled statistics into a fixed-length vector
        pooled_features = torch.cat([n_mean, n_max, n_std, b_mean, b_max, b_std], dim=0)
        
        # Pass through MLP
        return self.net(pooled_features)

In [40]:
readout = MLP(feature_dim=9)
print(readout)

MLP(
  (net): Sequential(
    (0): Linear(in_features=9, out_features=64, bias=True)
    (1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (2): SiLU()
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (6): SiLU()
    (7): Dropout(p=0.1, inplace=False)
    (8): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [47]:
mysteryTensors = (nodeFeatures, bondFeatures)
mysteryValue = readout(*mysteryTensors)
mysteryValue

tensor([0.1076], grad_fn=<ViewBackward0>)

## Now We Combine Everything

In [52]:
class QMPNN(nn.Module):
    def __init__(
        self,
        max_qubits: int = 16,
        max_bonds: int = 120,
        num_layers: int = 4,
        feature_dim: int = 9,      # 3*(2 node feats) + 3*(1 bond feat)
        hidden_dim: int = 64,
        dropout: float = 0.1
    ):
        super().__init__()
        self.max_qubits = max_qubits
        self.max_bonds = max_bonds
        self.num_layers = num_layers

        # Register quantum message-passing parameters
        self.quantum_params = nn.Parameter(
            torch.randn(num_layers, max_qubits + max_bonds, dtype=torch.float32)
        )

        # Predefine one device and QNode for the maximum size
        self.dev = qml.device("default.qubit", wires=max_qubits)

        @qml.qnode(self.dev, interface="torch")
        def _quantum_forward(matrix, edge_list, n_qubits, n_bonds):
            # 1) QMSE encoding on active qubits
            for i in range(n_qubits):
                qml.RY(matrix[i, i], wires=i)
            # 2) Initial bond rotations
            for i, j, _, _ in edge_list:
                qml.IsingXX(matrix[i, j], wires=[i, j])
            # 3) Message-passing layers
            for layer in range(self.num_layers):
                for edge_idx, (i, j, _, _) in enumerate(edge_list):
                    qml.IsingXX(self.quantum_params[layer, edge_idx], wires=[i, j])
                for i in range(n_qubits):
                    param_idx = len(edge_list) + i
                    qml.RY(self.quantum_params[layer, param_idx], wires=i)
            # 4) Gather fixed-length node & bond expectation lists
            node_feats = []
            for i in range(self.max_qubits):
                if i < n_qubits:
                    node_feats.append(qml.expval(qml.PauliZ(i)))
                    node_feats.append(qml.expval(qml.PauliX(i)))
                else:
                    node_feats.extend([0.0, 0.0])
            bond_feats = []
            for b in range(self.max_bonds):
                if b < n_bonds:
                    i, j, _, _ = edge_list[b]
                    bond_feats.append(qml.expval(qml.PauliZ(i) @ qml.PauliZ(j)))
                else:
                    bond_feats.append(0.0)
            return node_feats, bond_feats

        self._quantum_forward = _quantum_forward

        # Classical MLP with mean/max/std pooling
        self.MLP = MLP(feature_dim, hidden_dim, dropout)

    def forward(self, matrix, edge_list):
        """
        matrix:  [n_qubits, n_qubits] QMSE matrix for this molecule
        edge_list: list of (i, j, order, stereo) for this molecule
        """
        n_qubits = matrix.shape[0]
        n_bonds = len(edge_list)

        # 1) Run the single QNode with actual sizes
        node_list, bond_list = self._quantum_forward(
            matrix, edge_list, n_qubits, n_bonds
        )

        # 2) Stack into fixed-size tensors
        node_feats = torch.stack(node_list).reshape(self.max_qubits, 2)
        bond_feats = torch.stack(bond_list).reshape(self.max_bonds, 1)

        # 3) Select only active entries to avoid padding bias
        active_node_feats = node_feats[:n_qubits]
        active_bond_feats = bond_feats[:n_bonds] if n_bonds > 0 else torch.zeros((0, 1))

        # 4) Pool and predict via MLP
        prediction = self.MLP(active_node_feats, active_bond_feats)
        return prediction


In [53]:
MessagePassingNetwork = QMPNN()
print(MessagePassingNetwork)

QMPNN(
  (MLP): MLP(
    (net): Sequential(
      (0): Linear(in_features=9, out_features=64, bias=True)
      (1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (2): SiLU()
      (3): Dropout(p=0.1, inplace=False)
      (4): Linear(in_features=64, out_features=32, bias=True)
      (5): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (6): SiLU()
      (7): Dropout(p=0.1, inplace=False)
      (8): Linear(in_features=32, out_features=1, bias=True)
    )
  )
)
